<a href="https://colab.research.google.com/github/akshatdalton/AI-Expert-Roadmap/blob/main/Q1_2019102024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd seam-carving-akshatdalton/

/content/seam-carving-akshatdalton


In [ ]:
!ls

images	README.md  src


In [80]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [81]:
bangalore_dancers = cv2.cvtColor(cv2.imread("./images/bangalore-dancers.jpg").copy(), cv2.COLOR_BGR2RGB)
cat = cv2.cvtColor(cv2.imread("./images/cat.jpg").copy(), cv2.COLOR_BGR2RGB)
chhota_bheem = cv2.cvtColor(cv2.imread("./images/chhota-bheem.jpg").copy(), cv2.COLOR_BGR2RGB)
iiit = cv2.cvtColor(cv2.imread("./images/iiit.jpg").copy(), cv2.COLOR_BGR2RGB)
# maldives = cv2.cvtColor(cv2.imread("./images/maldives.jpg").copy(), cv2.COLOR_BGR2RGB)
path = cv2.cvtColor(cv2.imread("./images/path.jpg").copy(), cv2.COLOR_BGR2RGB)
shadow = cv2.cvtColor(cv2.imread("./images/shadow.jpg").copy(), cv2.COLOR_BGR2RGB)
tanjore = cv2.cvtColor(cv2.imread("./images/tanjore.jpg").copy(), cv2.COLOR_BGR2RGB)
trafalgar_square = cv2.cvtColor(cv2.imread("./images/trafalgar-square.jpg").copy(), cv2.COLOR_BGR2RGB)
uluru = cv2.cvtColor(cv2.imread("./images/uluru.jpg").copy(), cv2.COLOR_BGR2RGB)

In [82]:
def generate_energy_map(img):
    energy_map = np.zeros((img.shape[0], img.shape[1]), dtype="float")

    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            energy = np.zeros(img.shape[2], dtype=np.float64)

            for channel in range(img.shape[2]):
                if row > 0:
                    energy[channel] += np.square(img[row][col][channel] - img[row - 1][col][channel])
                if row + 1 < img.shape[0]:
                    energy[channel] += np.square(img[row][col][channel] - img[row + 1][col][channel])
                if col > 0:
                    energy[channel] += np.square(img[row][col][channel] - img[row][col - 1][channel])
                if col + 1 < img.shape[1]:
                    energy[channel] += np.square(img[row][col][channel] - img[row][col + 1][channel])

            energy_map[row][col] += np.sqrt(np.sum(energy))

    return energy_map

In [83]:
def remove_seam(img):
    result = np.empty(shape=(img.shape[0], img.shape[1] - 1, img.shape[2]), dtype="int")
    energy_map = generate_energy_map(img)
    dp = np.empty((energy_map.shape[0], energy_map.shape[1]), dtype="float")
    dp[0] = energy_map[0]

    for row in range(1, energy_map.shape[0]):
        for col in range(energy_map.shape[1]):
            dp[row][col] = energy_map[row][col] + energy_map[row - 1][col]
            if col > 0:
                dp[row][col] = min(dp[row][col], energy_map[row][col] + energy_map[row - 1][col - 1])
            if col + 1 < energy_map.shape[1]:
                dp[row][col] = min(dp[row][col], energy_map[row][col] + energy_map[row - 1][col + 1])

    last_index = dp[-1].argmin()
    for idx in range(dp.shape[0] - 1, -1, -1):
        current_min = dp[idx][last_index]
        delete_index = last_index

        if last_index > 0 and dp[idx][last_index - 1] < current_min:
            current_min = dp[idx][last_index - 1]
            delete_index = last_index - 1
        
        if last_index + 1 < dp.shape[1] and dp[idx][last_index + 1] < current_min:
            current_min = dp[idx][last_index + 1]
            delete_index = last_index + 1

        result[idx] = np.delete(img[idx], delete_index, axis=0)
        last_index = delete_index

    return result

In [84]:
import time

def content_aware_image_resizing(img, take_transpose=True):
    start = time.time()
    img = img.astype("int")
    shrinked_img = img

    plt.subplot(1, 4, 1)
    plt.imshow(img)
    plt.title("Original Image")
    plt.figure()
    plt.show()
    
    if take_transpose:
        shrinked_img = cv2.transpose(shrinked_img)

    for _ in range(int(0.2 * img.shape[1])):
        # 20% shrinkage
        shrinked_img = remove_seam(shrinked_img)
    
    if take_transpose:
        shrinked_img = cv2.transpose(shrinked_img)
    
    print("Shrinked Image dimension = ", shrinked_img.shape)
    print("Execution time for 20% = ", time.time() - start)
    
    plt.subplot(1, 4, 2)
    plt.imshow(shrinked_img)
    plt.title("20% shrinkage")
    plt.figure()
    plt.show()
    
    if take_transpose:
        shrinked_img = cv2.transpose(shrinked_img)
    
    for _ in range(int(0.2 * img.shape[1])):
        # Another 20% shrinkage => 40% shrinkage
        shrinked_img = remove_seam(shrinked_img)

    if take_transpose:
        shrinked_img = cv2.transpose(shrinked_img)
    
    print("Shrinked Image dimension = ", shrinked_img.shape)
    print("Execution time for 40% = ", time.time() - start)
    
    plt.subplot(1, 4, 3)
    plt.imshow(shrinked_img)
    plt.title("40% shrinkage")
    plt.figure()
    plt.show()
    
    if take_transpose:
        shrinked_img = cv2.transpose(shrinked_img)
    
    for _ in range(int(0.2 * img.shape[1])):
        # Another 20% shrinkage => 60% shrinkage
        shrinked_img = remove_seam(shrinked_img)
    
    if take_transpose:
        shrinked_img = cv2.transpose(shrinked_img)
    
    print("Shrinked Image dimension = ", shrinked_img.shape)
    print("Execution time for 60% = ", time.time() - start, "\n")

    plt.subplot(1, 4, 4)
    plt.imshow(shrinked_img)
    plt.title("60% shrinkage")
    plt.figure()
    plt.show()

In [ ]:
image_list = [bangalore_dancers, cat, iiit, shadow, trafalgar_square, uluru, tanjore]

for image in image_list:
    content_aware_image_resizing(image)

(378, 504, 3)
(150, 150, 3)
(1200, 1920, 3)
(369, 880, 3)
(333, 500, 3)
(500, 770, 3)
(319, 359, 3)
(378, 504, 3)
